In [87]:
import pandas as pd
import os

## A function used to update Country names cleanly is imported from another file in this project
from aux_fix_country_utils import fix_country_from_text

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)

In [6]:
DF_input_base = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\parsedXMLs_base\


In [7]:
DF_input_match_GLOBOCAN = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\parsedXMLs_match_GLOBOCAN\


## Note
The following corrections are performed afer separating the articles' dataset in two copies, one in which the cancer and country names are untouched (base) and another in which cancer and country names are modified to match those in the Globocan dataset. Here corrections in both copies are performed.

### Correcting *parsedX_100000.csv*

In [285]:
# Read csv
df_parsedX_100000_base = pd.read_csv(DF_input_base + "parsedX_100000.csv")
df_parsedX_100000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_100000.csv")

In [287]:
## Replace found errors by true Country values

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1522964, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1522964, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1318271, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1318271, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1525053, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1525053, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1586297, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1586297, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1703193, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1703193, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1768942, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1768942, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1772998, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1772998, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1874576, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1874576, ["Country", "Country_source"]] = fix_country_from_text("France")

#df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [288]:
df_parsedX_100000_base.to_csv(DF_input_base + "parsedX_100000.csv", index = False)
df_parsedX_100000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_100000.csv", index = False)

del df_parsedX_100000_base
del df_parsedX_100000_match_GLOBOCAN

### Correcting *parsedX_1000000.csv*

In [289]:
# Read csv
df_parsedX_1000000_base = pd.read_csv(DF_input_base + "parsedX_1000000.csv")
df_parsedX_1000000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1000000.csv")

In [290]:
## Replace found errors by true Country values

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11721110, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11721110, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11309027, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11309027, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11450918, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11450918, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11468033, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11468033, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11593153, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11593153, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11680021, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11680021, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11917379, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11917379, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11956310, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11956310, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 12016415, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 12016415, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 12058131, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 12058131, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 12108281, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 12108281, ["Country", "Country_source"]] = fix_country_from_text("France")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [291]:
df_parsedX_1000000_base.to_csv(DF_input_base + "parsedX_1000000.csv", index = False)
df_parsedX_1000000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1000000.csv", index = False)

del df_parsedX_1000000_base
del df_parsedX_1000000_match_GLOBOCAN

### Correcting *parsedX_1100000.csv*

In [292]:
# Read csv
df_parsedX_1100000_base = pd.read_csv(DF_input_base + "parsedX_1100000.csv")
df_parsedX_1100000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1100000.csv")

In [293]:
## Replace found errors by true Country values

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12957547, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12957547, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12355183, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12355183, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12161378, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12161378, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12506764, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12506764, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12621853, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12621853, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12422048, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12422048, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12538984, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12538984, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12873172, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12873172, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12145848, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12145848, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12205789, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12205789, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12846424, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12846424, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12972519, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12972519, ["Country", "Country_source"]] = fix_country_from_text("France")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [294]:
df_parsedX_1100000_base.to_csv(DF_input_base + "parsedX_1100000.csv", index = False)
df_parsedX_1100000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1100000.csv", index = False)

del df_parsedX_1100000_base
del df_parsedX_1100000_match_GLOBOCAN

### Correcting *parsedX_1200000.csv*

In [295]:
# Read csv
df_parsedX_1200000_base = pd.read_csv(DF_input_base + "parsedX_1200000.csv")
df_parsedX_1200000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1200000.csv")

In [296]:
## Replace found errors by true Country values

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15481700, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15481700, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14696430, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14696430, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15067206, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15067206, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14586258, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14586258, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14600589, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14600589, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14968015, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14968015, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15022781, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15022781, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15027374, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15027374, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15036243, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15036243, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15039608, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15039608, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15204374, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15204374, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15204423, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15204423, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15351012, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15351012, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15357755, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15357755, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15365563, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15365563, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15380568, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15380568, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15545939, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15545939, ["Country", "Country_source"]] = fix_country_from_text("France")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [298]:
df_parsedX_1200000_base.to_csv(DF_input_base + "parsedX_1200000.csv", index = False)
df_parsedX_1200000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1200000.csv", index = False)

del df_parsedX_1200000_base
del df_parsedX_1200000_match_GLOBOCAN

NameError: name 'df_parsedX_1200000_base' is not defined

### Correcting *parsedX_1300000.csv*

In [299]:
# Read csv
df_parsedX_1300000_base = pd.read_csv(DF_input_base + "parsedX_1300000.csv")
df_parsedX_1300000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1300000.csv")

In [300]:
## Replace found errors by true Country values

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16267798, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16267798, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15942562, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15942562, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15692413, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15692413, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15731292, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15731292, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15734139, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15734139, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15748131, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15748131, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15808448, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15808448, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15810157, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15810157, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16039103, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16039103, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16129520, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16129520, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16134302, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16134302, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16181454, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16181454, ["Country", "Country_source"]] = fix_country_from_text("Australia")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [301]:
df_parsedX_1300000_base.to_csv(DF_input_base + "parsedX_1300000.csv", index = False)
df_parsedX_1300000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1300000.csv", index = False)

del df_parsedX_1300000_base
del df_parsedX_1300000_match_GLOBOCAN

### Correcting *parsedX_1400000.csv*

In [302]:
# Read csv
df_parsedX_1400000_base = pd.read_csv(DF_input_base + "parsedX_1400000.csv")
df_parsedX_1400000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1400000.csv")

In [303]:
## Replace found errors by true Country values

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16715323, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16715323, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17007025, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17007025, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16340834, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16340834, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16482873, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16482873, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16642235, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16642235, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16697618, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16697618, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17030388, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17030388, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17091098, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17091098, ["Country", "Country_source"]] = fix_country_from_text("Australia")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [304]:
df_parsedX_1400000_base.to_csv(DF_input_base + "parsedX_1400000.csv", index = False)
df_parsedX_1400000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1400000.csv", index = False)

del df_parsedX_1400000_base
del df_parsedX_1400000_match_GLOBOCAN

### Correcting *parsedX_1500000.csv*

In [305]:
# Read csv
df_parsedX_1500000_base = pd.read_csv(DF_input_base + "parsedX_1500000.csv")
df_parsedX_1500000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1500000.csv")

In [306]:
## Replace found errors by true Country values

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17406831, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17406831, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17927986, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17927986, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17383523, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17383523, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17389413, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17389413, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17479108, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17479108, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17596859, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17596859, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17702593, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17702593, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17932053, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17932053, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 18080419, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 18080419, ["Country", "Country_source"]] = fix_country_from_text("France")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [307]:
df_parsedX_1500000_base.to_csv(DF_input_base + "parsedX_1500000.csv", index = False)
df_parsedX_1500000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1500000.csv", index = False)

del df_parsedX_1500000_base
del df_parsedX_1500000_match_GLOBOCAN

### Correcting *parsedX_1600000.csv*

In [309]:
# Read csv
df_parsedX_1600000_base = pd.read_csv(DF_input_base + "parsedX_1600000.csv")
df_parsedX_1600000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1600000.csv")

In [310]:
## Replace found errors by true Country values

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18228178, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18228178, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 19011482, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 19011482, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18365760, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18365760, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18977715, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18977715, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18408385, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18408385, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18380311, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18380311, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18456062, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18456062, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18538271, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18538271, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18557535, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18557535, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18588522, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18588522, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18604657, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18604657, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18656282, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18656282, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18807605, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18807605, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 19031932, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 19031932, ["Country", "Country_source"]] = fix_country_from_text("Austria")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [311]:
df_parsedX_1600000_base.to_csv(DF_input_base + "parsedX_1600000.csv", index = False)
df_parsedX_1600000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1600000.csv", index = False)

del df_parsedX_1600000_base
del df_parsedX_1600000_match_GLOBOCAN

### Correcting *parsedX_1700000.csv*

In [312]:
# Read csv
df_parsedX_1700000_base = pd.read_csv(DF_input_base + "parsedX_1700000.csv")
df_parsedX_1700000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1700000.csv")

In [313]:
## Replace found errors by true Country values

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19046130, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19046130, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19051189, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19051189, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19061466, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19061466, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19075591, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19075591, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19126291, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19126291, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19168018, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19168018, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19178759, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19178759, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19268208, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19268208, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19326171, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19326171, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19479418, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19479418, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19647630, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19647630, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19760114, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19760114, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19801219, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19801219, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19824281, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19824281, ["Country", "Country_source"]] = fix_country_from_text("Australia")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [314]:
df_parsedX_1700000_base.to_csv(DF_input_base + "parsedX_1700000.csv", index = False)
df_parsedX_1700000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1700000.csv", index = False)

del df_parsedX_1700000_base
del df_parsedX_1700000_match_GLOBOCAN

### Correcting *parsedX_1800000.csv*

In [315]:
# Read csv
df_parsedX_1800000_base = pd.read_csv(DF_input_base + "parsedX_1800000.csv")
df_parsedX_1800000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1800000.csv")

In [316]:
## Replace found errors by true Country values

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20704980, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20704980, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20116771, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20116771, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19858739, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19858739, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19858743, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19858743, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20651972, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20651972, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20079483, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20079483, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20177103, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20177103, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20399038, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20399038, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20423029, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20423029, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20423030, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20423030, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20423034, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20423034, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20423041, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20423041, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20594856, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20594856, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20607507, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20607507, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20690526, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20690526, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [317]:
df_parsedX_1800000_base.to_csv(DF_input_base + "parsedX_1800000.csv", index = False)
df_parsedX_1800000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1800000.csv", index = False)

del df_parsedX_1800000_base
del df_parsedX_1800000_match_GLOBOCAN

### Correcting *parsedX_1900000.csv*

In [318]:
# Read csv
df_parsedX_1900000_base = pd.read_csv(DF_input_base + "parsedX_1900000.csv")
df_parsedX_1900000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1900000.csv")

In [319]:
## Replace found errors by true Country values

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20979677, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20979677, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20981674, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20981674, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21353748, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21353748, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21389920, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21389920, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21513998, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21513998, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21577052, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21577052, ["Country", "Country_source"]] = fix_country_from_text("Australia")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [320]:
df_parsedX_1900000_base.to_csv(DF_input_base + "parsedX_1900000.csv", index = False)
df_parsedX_1900000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1900000.csv", index = False)

del df_parsedX_1900000_base
del df_parsedX_1900000_match_GLOBOCAN

### Correcting *parsedX_200000.csv*

In [321]:
# Read csv
df_parsedX_200000_base = pd.read_csv(DF_input_base + "parsedX_200000.csv")
df_parsedX_200000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_200000.csv")

In [ ]:
## Replace found errors by true Country values

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998100, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998100, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [ ]:
df_parsedX_200000_base.to_csv(DF_input_base + "parsedX_200000.csv", index = False)
df_parsedX_200000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_200000.csv", index = False)

del df_parsedX_200000_base
del df_parsedX_200000_match_GLOBOCAN

### Correcting *parsedX_2100000.csv*

In [222]:
# Read csv
df_parsedX_2100000_base = pd.read_csv(DF_input_base + "parsedX_2100000.csv")
df_parsedX_2100000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2100000.csv")

In [224]:
## Replace found errors by true Country values

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22504321, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22504321, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23150848, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23150848, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [225]:
df_parsedX_2100000_base.to_csv(DF_input_base + "parsedX_2100000.csv", index = False)
df_parsedX_2100000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2100000.csv", index = False)

del df_parsedX_2100000_base
del df_parsedX_2100000_match_GLOBOCAN

### Correcting *parsedX_2300000.csv*

In [50]:
# Read csv
df_parsedX_2300000_base = pd.read_csv(DF_input_base + "parsedX_2300000.csv")
df_parsedX_2300000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2300000.csv")

C:\Users\svalb\AppData\Local\Temp\ipykernel_53352\2131202583.py:2: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_parsedX_2300000_base = pd.read_csv(DF_input_base + "parsedX_2300000.csv")
C:\Users\svalb\AppData\Local\Temp\ipykernel_53352\2131202583.py:3: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_parsedX_2300000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2300000.csv")


In [51]:
## Replace found errors by true Country values

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24198623, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24198623, ["Country", "Country_source"]] = fix_country_from_text("Austria")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [52]:
df_parsedX_2300000_base.to_csv(DF_input_base + "parsedX_2300000.csv", index = False)
df_parsedX_2300000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2300000.csv", index = False)

del df_parsedX_2300000_base
del df_parsedX_2300000_match_GLOBOCAN

### Correcting *parsedX_2400000.csv*

In [226]:
# Read csv
df_parsedX_2400000_base = pd.read_csv(DF_input_base + "parsedX_2400000.csv")
df_parsedX_2400000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2400000.csv")

In [227]:
## Replace found errors by true Country values

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24781824, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24781824, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25019046, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25019046, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25253429, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25253429, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24739432, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24739432, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25010257, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25010257, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [228]:
df_parsedX_2400000_base.to_csv(DF_input_base + "parsedX_2400000.csv", index = False)
df_parsedX_2400000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2400000.csv", index = False)

del df_parsedX_2400000_base
del df_parsedX_2400000_match_GLOBOCAN

### Correcting *parsedX_2500000.csv*

In [230]:
# Read csv
df_parsedX_2500000_base = pd.read_csv(DF_input_base + "parsedX_2500000.csv")
df_parsedX_2500000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2500000.csv")

In [231]:
## Replace found errors by true Country values

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25936229, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25936229, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25575781, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25575781, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25569638, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25569638, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25420684, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25420684, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25420684, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25634551, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [232]:
df_parsedX_2500000_base.to_csv(DF_input_base + "parsedX_2500000.csv", index = False)
df_parsedX_2500000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2500000.csv", index = False)

del df_parsedX_2500000_base
del df_parsedX_2500000_match_GLOBOCAN

### Correcting *parsedX_2600000.csv*

In [233]:
# Read csv
df_parsedX_2600000_base = pd.read_csv(DF_input_base + "parsedX_2600000.csv")
df_parsedX_2600000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2600000.csv")

In [234]:
## Replace found errors by true Country values

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26047983, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26047983, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26149098, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26149098, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [235]:
df_parsedX_2600000_base.to_csv(DF_input_base + "parsedX_2600000.csv", index = False)
df_parsedX_2600000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2600000.csv", index = False)

del df_parsedX_2600000_base
del df_parsedX_2600000_match_GLOBOCAN

### Correcting *parsedX_2700000.csv*

In [236]:
# Read csv
df_parsedX_2700000_base = pd.read_csv(DF_input_base + "parsedX_2700000.csv")
df_parsedX_2700000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2700000.csv")

In [237]:
## Replace found errors by true Country values

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26974049, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26974049, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27292875, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27292875, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [238]:
df_parsedX_2700000_base.to_csv(DF_input_base + "parsedX_2700000.csv", index = False)
df_parsedX_2700000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2700000.csv", index = False)

del df_parsedX_2700000_base
del df_parsedX_2700000_match_GLOBOCAN

### Correcting *parsedX_2800000.csv*

In [239]:
# Read csv
df_parsedX_2800000_base = pd.read_csv(DF_input_base + "parsedX_2800000.csv")
df_parsedX_2800000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2800000.csv")

In [240]:
## Replace found errors by true Country values

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28198377, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28198377, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27665490, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27665490, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [241]:
df_parsedX_2800000_base.to_csv(DF_input_base + "parsedX_2800000.csv", index = False)
df_parsedX_2800000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2800000.csv", index = False)

del df_parsedX_2800000_base
del df_parsedX_2800000_match_GLOBOCAN

### Correcting *parsedX_2900000.csv*

In [242]:
# Read csv
df_parsedX_2900000_base = pd.read_csv(DF_input_base + "parsedX_2900000.csv")
df_parsedX_2900000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2900000.csv")

In [243]:
## Replace found errors by true Country values

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28499594, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28499594, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28719413, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28719413, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [244]:
df_parsedX_2900000_base.to_csv(DF_input_base + "parsedX_2900000.csv", index = False)
df_parsedX_2900000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2900000.csv", index = False)

del df_parsedX_2900000_base
del df_parsedX_2900000_match_GLOBOCAN

### Correcting *parsedX_3000000.csv*

In [245]:
# Read csv
df_parsedX_3000000_base = pd.read_csv(DF_input_base + "parsedX_3000000.csv")
df_parsedX_3000000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3000000.csv")

In [246]:
## Replace found errors by true Country values

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29427708, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29427708, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28943004, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28943004, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29408798, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29408798, ["Country", "Country_source"]] = fix_country_from_text("India")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [247]:
df_parsedX_3000000_base.to_csv(DF_input_base + "parsedX_3000000.csv", index = False)
df_parsedX_3000000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3000000.csv", index = False)

del df_parsedX_3000000_base
del df_parsedX_3000000_match_GLOBOCAN

### Correcting *parsedX_3100000.csv*

In [248]:
# Read csv
df_parsedX_3100000_base = pd.read_csv(DF_input_base + "parsedX_3100000.csv")
df_parsedX_3100000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3100000.csv")

In [249]:
## Replace found errors by true Country values

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29856243, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29856243, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29653120, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29653120, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30229205, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30229205, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [250]:
df_parsedX_3100000_base.to_csv(DF_input_base + "parsedX_3100000.csv", index = False)
df_parsedX_3100000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3100000.csv", index = False)

del df_parsedX_3100000_base
del df_parsedX_3100000_match_GLOBOCAN

### Correcting *parsedX_3200000.csv*

In [251]:
# Read csv
df_parsedX_3200000_base = pd.read_csv(DF_input_base + "parsedX_3200000.csv")
df_parsedX_3200000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3200000.csv")

In [252]:
## Replace found errors by true Country values

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30585274, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30585274, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30318359, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30318359, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30770107, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30770107, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [253]:
df_parsedX_3200000_base.to_csv(DF_input_base + "parsedX_3200000.csv", index = False)
df_parsedX_3200000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3200000.csv", index = False)

del df_parsedX_3200000_base
del df_parsedX_3200000_match_GLOBOCAN

### Correcting *parsedX_3300000.csv*

In [254]:
# Read csv
df_parsedX_3300000_base = pd.read_csv(DF_input_base + "parsedX_3300000.csv")
df_parsedX_3300000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3300000.csv")

In [255]:
## Replace found errors by true Country values

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 30954945, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 30954945, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31324932, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31324932, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31107807, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31107807, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [256]:
df_parsedX_3300000_base.to_csv(DF_input_base + "parsedX_3300000.csv", index = False)
df_parsedX_3300000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3300000.csv", index = False)

del df_parsedX_3300000_base
del df_parsedX_3300000_match_GLOBOCAN

### Correcting *parsedX_3400000.csv*

In [53]:
# Read csv
df_parsedX_3400000_base = pd.read_csv(DF_input_base + "parsedX_3400000.csv")
df_parsedX_3400000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3400000.csv")

C:\Users\svalb\AppData\Local\Temp\ipykernel_53352\501477961.py:2: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_parsedX_3400000_base = pd.read_csv(DF_input_base + "parsedX_3400000.csv")
C:\Users\svalb\AppData\Local\Temp\ipykernel_53352\501477961.py:3: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_parsedX_3400000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3400000.csv")


In [54]:
## Replace found errors by true Country values

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31889898, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31889898, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31610880, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31610880, ["Country", "Country_source"]] = fix_country_from_text("Austria")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [55]:
df_parsedX_3400000_base.to_csv(DF_input_base + "parsedX_3400000.csv", index = False)
df_parsedX_3400000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3400000.csv", index = False)

del df_parsedX_3400000_base
del df_parsedX_3400000_match_GLOBOCAN

### Correcting *parsedX_3500000.csv*

In [257]:
# Read csv
df_parsedX_3500000_base = pd.read_csv(DF_input_base + "parsedX_3500000.csv")
df_parsedX_3500000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3500000.csv")

In [258]:
## Replace found errors by true Country values

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32719975, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32719975, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32192726, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32192726, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32522324, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32522324, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32734609, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32734609, ["Country", "Country_source"]] = fix_country_from_text("India")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [259]:
df_parsedX_3500000_base.to_csv(DF_input_base + "parsedX_3500000.csv", index = False)
df_parsedX_3500000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3500000.csv", index = False)

del df_parsedX_3500000_base
del df_parsedX_3500000_match_GLOBOCAN

### Correcting *parsedX_3600000.csv*

In [260]:
# Read csv
df_parsedX_3600000_base = pd.read_csv(DF_input_base + "parsedX_3600000.csv")
df_parsedX_3600000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3600000.csv")

In [261]:
## Replace found errors by true Country values

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33073332, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33073332, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33334598, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33334598, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [262]:
df_parsedX_3600000_base.to_csv(DF_input_base + "parsedX_3600000.csv", index = False)
df_parsedX_3600000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3600000.csv", index = False)

del df_parsedX_3600000_base
del df_parsedX_3600000_match_GLOBOCAN

### Correcting *parsedX_3700000.csv*

In [263]:
# Read csv
df_parsedX_3700000_base = pd.read_csv(DF_input_base + "parsedX_3700000.csv")
df_parsedX_3700000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3700000.csv")

In [264]:
## Replace found errors by true Country values

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33749551, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33749551, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33848668, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33848668, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33903933, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33903933, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33616840, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33616840, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33551073, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33551073, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [265]:
df_parsedX_3700000_base.to_csv(DF_input_base + "parsedX_3700000.csv", index = False)
df_parsedX_3700000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3700000.csv", index = False)

del df_parsedX_3700000_base
del df_parsedX_3700000_match_GLOBOCAN

### Correcting *parsedX_3800000.csv*

In [166]:
# Read csv
df_parsedX_3800000_base = pd.read_csv(DF_input_base + "parsedX_3800000.csv")
df_parsedX_3800000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3800000.csv")

In [167]:
## Replace found errors by true Country values

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34414870, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34414870, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34374389, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34374389, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34185219, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34185219, ["Country", "Country_source"]] = fix_country_from_text("Austria")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [168]:
df_parsedX_3800000_base.to_csv(DF_input_base + "parsedX_3800000.csv", index = False)
df_parsedX_3800000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3800000.csv", index = False)

del df_parsedX_3800000_base
del df_parsedX_3800000_match_GLOBOCAN

### Correcting *parsedX_3900000.csv*

In [266]:
# Read csv
df_parsedX_3900000_base = pd.read_csv(DF_input_base + "parsedX_3900000.csv")
df_parsedX_3900000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3900000.csv")

In [267]:
## Replace found errors by true Country values

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34937625, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34937625, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [268]:
df_parsedX_3900000_base.to_csv(DF_input_base + "parsedX_3900000.csv", index = False)
df_parsedX_3900000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3900000.csv", index = False)

del df_parsedX_3900000_base
del df_parsedX_3900000_match_GLOBOCAN

### Correcting *parsedX_4200000.csv*

In [269]:
# Read csv
df_parsedX_4200000_base = pd.read_csv(DF_input_base + "parsedX_4200000.csv")
df_parsedX_4200000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_4200000.csv")

In [270]:
## Replace found errors by true Country values

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36994202, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36994202, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37096110, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37096110, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [271]:
df_parsedX_4200000_base.to_csv(DF_input_base + "parsedX_4200000.csv", index = False)
df_parsedX_4200000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_4200000.csv", index = False)

del df_parsedX_4200000_base
del df_parsedX_4200000_match_GLOBOCAN

### Correcting *parsedX_4400000.csv*

In [84]:
# Read csv
df_parsedX_4400000_base = pd.read_csv(DF_input_base + "parsedX_4400000.csv")
df_parsedX_4400000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_4400000.csv")

C:\Users\svalb\AppData\Local\Temp\ipykernel_53352\688935312.py:2: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_parsedX_4400000_base = pd.read_csv(DF_input_base + "parsedX_4400000.csv")
C:\Users\svalb\AppData\Local\Temp\ipykernel_53352\688935312.py:3: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_parsedX_4400000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_4400000.csv")


In [85]:
## Replace found errors by true Country values

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38527596, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38527596, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")


In [86]:
df_parsedX_4400000_base.to_csv(DF_input_base + "parsedX_4400000.csv", index = False)
df_parsedX_4400000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_4400000.csv", index = False)

del df_parsedX_4400000_base
del df_parsedX_4400000_match_GLOBOCAN

### Correcting *parsedX_4454000.csv*

In [272]:
# Read csv
df_parsedX_4454000_base = pd.read_csv(DF_input_base + "parsedX_4454000.csv")
df_parsedX_4454000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_4454000.csv")

In [273]:
## Replace found errors by true Country values

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38750928, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38750928, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38903136, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38903136, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [274]:
df_parsedX_4454000_base.to_csv(DF_input_base + "parsedX_4454000.csv", index = False)
df_parsedX_4454000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_4454000.csv", index = False)

del df_parsedX_4454000_base
del df_parsedX_4454000_match_GLOBOCAN

### Correcting *parsedX_500000.csv*

In [275]:
# Read csv
df_parsedX_500000_base = pd.read_csv(DF_input_base + "parsedX_500000.csv")
df_parsedX_500000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_500000.csv")

In [276]:
## Replace found errors by true Country values

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7716605, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7716605, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [278]:
df_parsedX_500000_base.to_csv(DF_input_base + "parsedX_500000.csv", index = False)
df_parsedX_500000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_500000.csv", index = False)

del df_parsedX_500000_base
del df_parsedX_500000_match_GLOBOCAN

### Correcting *parsedX_600000.csv*

In [169]:
# Read csv
df_parsedX_600000_base = pd.read_csv(DF_input_base + "parsedX_600000.csv")
df_parsedX_600000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_600000.csv")

In [170]:
## Replace found errors by true Country values

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8671916, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8671916, ["Country", "Country_source"]] = fix_country_from_text("Austria")

#df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [171]:
df_parsedX_600000_base.to_csv(DF_input_base + "parsedX_600000.csv", index = False)
df_parsedX_600000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_600000.csv", index = False)

del df_parsedX_600000_base
del df_parsedX_600000_match_GLOBOCAN

### Correcting *parsedX_700000.csv*

In [279]:
# Read csv
df_parsedX_700000_base = pd.read_csv(DF_input_base + "parsedX_700000.csv")
df_parsedX_700000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_700000.csv")

In [280]:
## Replace found errors by true Country values

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9312254, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9312254, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9413693, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9413693, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [281]:
df_parsedX_700000_base.to_csv(DF_input_base + "parsedX_700000.csv", index = False)
df_parsedX_700000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_700000.csv", index = False)

del df_parsedX_700000_base
del df_parsedX_700000_match_GLOBOCAN

### Correcting *parsedX_800000.csv*

In [41]:
# Read csv
df_parsedX_800000_base = pd.read_csv(DF_input_base + "parsedX_800000.csv")
df_parsedX_800000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_800000.csv")

In [42]:
## Replace found errors by true Country values

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9589843, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9589843, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [43]:
df_parsedX_800000_base.to_csv(DF_input_base + "parsedX_800000.csv", index = False)
df_parsedX_800000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_800000.csv", index = False)

del df_parsedX_800000_base
del df_parsedX_800000_match_GLOBOCAN

### Correcting *parsedX_900000.csv*

In [282]:
# Read csv
df_parsedX_900000_base = pd.read_csv(DF_input_base + "parsedX_900000.csv")
df_parsedX_900000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_900000.csv")

In [283]:
## Replace found errors by true Country values

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10791581, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10791581, ["Country", "Country_source"]] = fix_country_from_text("United States")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [284]:
df_parsedX_900000_base.to_csv(DF_input_base + "parsedX_900000.csv", index = False)
df_parsedX_900000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_900000.csv", index = False)

del df_parsedX_900000_base
del df_parsedX_900000_match_GLOBOCAN